<a href="https://colab.research.google.com/github/jaspreetcodes/Mini-Projects/blob/main/fakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install simpletransformers tqdm pandas

In [ ]:
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
from simpletransformers.t5 import T5Model
import sklearn
import os
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
# preprocess the data and create the dataset splits

train = pd.read_csv("train_20000.csv") # read te dataset
val = pd.read_csv("val.csv") # read the dataset
test = pd.read_csv("test.csv") # read the dataset


lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"")
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)

    # do we perform the preprocessing steps?
    #filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    #stem_words=[stemmer.stem(w) for w in filtered_words]
    #lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(tokens) # or filter_words

train['cleanText']=train['text'].map(lambda s:preprocess(s))
train = train[['cleanText', 'label']]


test['cleanText']=test['text'].map(lambda s:preprocess(s))
test = test[['cleanText', 'label']]


val['cleanText']=val['text'].map(lambda s:preprocess(s))
val = val[['cleanText', 'label']]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:

# check the data

print(train.shape)
print(len(train[train['label']==0]))

#val.shape
#test['cleanText'].iloc[1]

(20000, 2)
9696


In [ ]:
# check the data

train.head(5)

,cleanText,label
0,the video from is a little blurry but the audi...,1
1,by james holbrooks a day after the administrat...,1
2,reuters u s democratic presidential candidate ...,0
3,maybe hillary or bernie can use this footage i...,1
4,главная новости впервые за лет сша направили п...,1


In [ ]:
# Renaming the columns
train.columns = ["input_text","target_text"]
val.columns = ["input_text","target_text"]

In [ ]:
model_args = ClassificationArgs(num_train_epochs=4, max_seq_length = 256, overwrite_output_dir=True, use_multiprocessing=False, use_multiprocessing_for_evaluation=False, process_count=1)

model = ClassificationModel(
    "bert", "bert-base-cased", args=model_args
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_args

ClassificationArgs(adafactor_beta1=None, adafactor_clip_threshold=1.0, adafactor_decay_rate=-0.8, adafactor_eps=(1e-30, 0.001), adafactor_relative_step=True, adafactor_scale_parameter=True, adafactor_warmup_init=True, adam_betas=(0.9, 0.999), adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', config={}, cosine_schedule_num_cycles=0.5, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=0, do_lower_case=False, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=100, evaluate_during_training=False, evaluate_during_training_silent=True, evaluate_during_training_steps=2000, evaluate_during_training_verbose=False, evaluate_each_epoch=True, fp16=True, gradient_accumulation_steps=1, learning_rate=4e-05, local_rank=-1, logging_steps=50, loss_type=None, loss_args={}, manual_see

In [ ]:
model.train_model(train, eval_data=val, use_cuda=True, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/2500 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/2500 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/2500 [00:00<?, ?it/s]

Running Epoch 4 of 4:   0%|          | 0/2500 [00:00<?, ?it/s]

(10000, 0.15960648785084486)

In [ ]:
# Predict full test set

test_df = test.filter(['cleanText', 'label'], axis=1)
test_df.columns = ["input_text","target_text"]

ref = []
to_predict = []
for i in test_df.index:
  input = test_df['input_text'][i]
  ref.append(test_df['target_text'][i])
  to_predict.append(input)
  #print(input)
  #pred.append(trained_model.predict(f"{prefix}: {input}"))

predictions = model.predict(to_predict)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(ref, predictions[0].tolist(), digits=3))


              precision    recall  f1-score   support

           0      0.961     0.933     0.947       480
           1      0.940     0.965     0.953       520

    accuracy                          0.950      1000
   macro avg      0.951     0.949     0.950      1000
weighted avg      0.950     0.950     0.950      1000

